# 构建神经网络
神经网络由对数据执行操作的层/模块组成。 torch.nn 命名空间提供了构建您自己的神经网络所需的所有构建块。PyTorch 中的每个模块都是 nn.Module 的子类。神经网络本身就是一个模块，它由其他模块（层）组成。这种嵌套结构允许轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络来对 FashionMNIST 数据集中的图像进行分类。

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 获取训练设备
我们希望能够在硬件加速器（如 GPU 或 MPS）上训练我们的模型，如果可用的话。让我们检查一下 torch.cuda 或 torch.backends.mps 是否可用，否则我们将使用 CPU。

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"device {device}")

device cpu


## 定义模型类
我们通过继承`nn.Module`来定义我们的神经网络，并在 `__init__` 中初始化神经网络层。每个 `nn.Module` 子类都在 `forward` 方法中实现对输入数据的操作。

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layout = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        out = self.layout(x)
        return out

我们创建一个NeuralNetwork实例并将其移动到device上：

In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layout): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用模型，我们将输入数据传递给它。这将执行模型的 forward 方法，以及一些 后台操作。不要直接调用 model.forward()！

在输入上调用模型会返回一个二维张量，其中 dim=0 对应于每个类的 10 个原始预测值的每个输出，dim=1 对应于每个输出的各个值。我们通过将其传递给 nn.Softmax 模块的实例来获得预测概率。

In [19]:
X = torch.rand(1, 28, 28, device=device)
# print(X)

out = model(X)
print(out)

pred_prob = nn.Softmax(dim=1)(out)
print(pred_prob)

y_pred = pred_prob.argmax(1)
print(y_pred)
print(f"Predicted class: {y_pred}")

tensor([[ 0.0806, -0.0381,  0.0002,  0.1662, -0.0061, -0.0794,  0.0175, -0.1399,
          0.0622,  0.0619]], grad_fn=<AddmmBackward0>)
tensor([[0.1067, 0.0947, 0.0984, 0.1162, 0.0978, 0.0909, 0.1002, 0.0856, 0.1047,
         0.1047]], grad_fn=<SoftmaxBackward0>)
tensor([3])
Predicted class: tensor([3])


## 模型层
让我们分解 FashionMNIST 模型中的层。为了说明这一点，我们将获取一个大小为 28x28 的 3 张图像的样本 minibatch，并查看当我们将其传递给网络时会发生什么。

In [20]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
我们初始化 nn.Flatten 层，将每个 2D 28x28 图像转换为 784 个像素值的连续数组（minibatch 维度（在 dim=0 处）保持不变）。

In [27]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

### nn.Linear
线性层 是一个模块，它使用其存储的权重和偏差对输入应用线性变换。

In [31]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
print(layer1)
hidden1 = layer1(flat_image)
print(hidden1.size())

Linear(in_features=784, out_features=20, bias=True)
torch.Size([3, 20])


### nn.ReLU
非线性激活函数是在模型的输入和输出之间创建复杂映射的原因。它们应用于线性变换之后以引入_非线性_，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用 nn.ReLU，但还有其他激活函数可以在你的模型中引入非线性。

In [32]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2990, -0.7515,  0.2314,  0.2621, -0.0281,  0.3658, -0.7137,  0.2839,
         -0.6108,  0.1043,  0.1417,  0.2000,  0.1022, -0.3582,  0.3667, -0.0601,
         -0.6928, -0.4936,  0.1503, -0.0134],
        [ 0.0666, -0.5275,  0.1474,  0.3404,  0.0647,  0.4545, -0.8173,  0.1863,
         -0.4470,  0.0584, -0.0381,  0.6315,  0.0937, -0.3427,  0.4257, -0.1828,
         -0.6695, -0.4099, -0.0665, -0.1068],
        [-0.0972, -0.6145, -0.0150,  0.0800, -0.0684,  0.6106, -0.9313,  0.0690,
         -0.7143,  0.0346,  0.1515,  0.4812,  0.3094, -0.1760,  0.2195, -0.1643,
         -0.8402, -0.3973,  0.0037,  0.2351]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2314, 0.2621, 0.0000, 0.3658, 0.0000, 0.2839, 0.0000,
         0.1043, 0.1417, 0.2000, 0.1022, 0.0000, 0.3667, 0.0000, 0.0000, 0.0000,
         0.1503, 0.0000],
        [0.0666, 0.0000, 0.1474, 0.3404, 0.0647, 0.4545, 0.0000, 0.1863, 0.0000,
         0.0584, 0.0000, 0.6315, 0.0937, 0.0000, 0.42

### nn.Sequential
nn.Sequential 是一个有序的模块容器。数据按定义的相同顺序传递给所有模块。你可以使用顺序容器来组合一个像 seq_modules 这样的快速网络。

In [33]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits.size())

torch.Size([3, 10])


### nn.softmax
神经网络的最后一个线性层返回_logits_ - [-infty, infty] 中的原始值 - 这些值被传递给 nn.Softmax 模块。logits 被缩放到 [0, 1] 之间的值，表示模型对每个类的预测概率。 dim 参数指示值必须沿其求和为 1 的维度。

In [37]:
res = nn.Softmax(dim=1)(logits)
print(res)

tensor([[0.0775, 0.1110, 0.1050, 0.0935, 0.1191, 0.0733, 0.0791, 0.0945, 0.1328,
         0.1143],
        [0.0739, 0.0959, 0.1058, 0.0931, 0.1407, 0.0627, 0.0717, 0.1000, 0.1236,
         0.1326],
        [0.0759, 0.1104, 0.0995, 0.0967, 0.1198, 0.0840, 0.0745, 0.1001, 0.1164,
         0.1226]], grad_fn=<SoftmaxBackward0>)


## 模型参数
神经网络中的许多层都是_参数化的_，即具有在训练期间优化的相关权重和偏差。继承 nn.Module 会自动跟踪模型对象中定义的所有字段，并可以使用模型的 parameters() 或 named_parameters() 方法访问所有参数。

In [38]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layout): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: layout.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 2.0048e-03,  3.0478e-02,  2.0277e-02,  ...,  3.2663e-03,
         -2.7711e-02,  1.4063e-02],
        [ 3.9998e-05, -1.3247e-02, -1.8281e-02,  ...,  6.2658e-03,
          2.2275e-02,  2.2839e-02]], grad_fn=<SliceBackward0>) 

Layer: layout.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0256,  0.0276], grad_fn=<SliceBackward0>) 

Layer: layout.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0429, -0.0170, -0.0161,  ...,  0.0203, -0.0123, -0.0330],
        [ 0.0011,  0.0134,  0.0193,  ...,  0.0180, -0.0062,  0.0281]],
       grad_fn=<SliceBackward0>) 

Layer: layout.2.bia